In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch

In [2]:
class CNN1DModel(nn.Module):
    def __init__(self, num_numerical_features, n_target,
                 symbol_size=39, f09_size=83, f10_size=13,
                 f11_size=540, time_size=967):
        super().__init__()

        # 임베딩 디멘션 설정
        self.embedding_dims = {
            'symbol': 8,
            'f09': 4,
            'f10': 4,
            'f11': 4,
            'time': 16
        }

        # 임베딩 레이어 초기화
        self.embeddings = nn.ModuleDict({
            'symbol': nn.Embedding(symbol_size, self.embedding_dims['symbol']),
            'f09': nn.Embedding(f09_size, self.embedding_dims['f09']),
            'f10': nn.Embedding(f10_size, self.embedding_dims['f10']),
            'f11': nn.Embedding(f11_size, self.embedding_dims['f11']),
            'time': nn.Embedding(time_size, self.embedding_dims['time'])
        })

        # 전체 임베딩 차원 계산
        self.total_embedding_dim = sum(self.embedding_dims.values())
        self.input_dim = self.total_embedding_dim + num_numerical_features

        # Dense layers with batch normalization
        self.dense1 = nn.Sequential(
            nn.Linear(self.input_dim, 258),
            nn.BatchNorm1d(258),
            nn.ReLU()
        )

        # Convolution blocks with residual connections
        self.conv1 = nn.Sequential(
            Conv1dBlock(128, 64, kernel_size=5),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.avg_pool = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)

        self.conv2 = nn.Sequential(
            Conv1dBlock(64, 32, kernel_size=3),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.max_pool = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        # Dropout for regularization
        self.dropout = nn.Dropout(0.2)

        # Final dense layers with batch normalization
        self.dense2 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            self.dropout
        )

        self.dense3 = nn.Sequential(
            nn.Linear(1024, n_target),
            nn.Tanh()
        )

    def forward(self, numerical_features, symbol, feature_09, feature_10, feature_11, time):
        # Embedding processing using ModuleDict
        embeddings = [
            self.embeddings['symbol'](symbol),
            self.embeddings['f09'](feature_09),
            self.embeddings['f10'](feature_10),
            self.embeddings['f11'](feature_11),
            self.embeddings['time'](time)
        ]

        # Concatenate all features
        x = torch.cat([numerical_features] + embeddings, dim=1)

        # Reshape for convolution
        x = x.unsqueeze(-1)  # Add channel dimension
        x = self.dense1(x)
        x = x.transpose(1, 2)  # (batch_size, channels, sequence_length)

        # Apply CNN layers with residual connections
        identity = x
        x = self.conv1(x)
        x = x + identity[:, :x.size(1), :x.size(2)]  # Residual connection
        x = self.avg_pool(x)

        identity = x
        x = self.conv2(x)
        x = x + identity[:, :x.size(1), :x.size(2)]  # Residual connection
        x = self.max_pool(x)

        # Final processing
        x = x.flatten(1)
        x = self.dense2(x)
        x = self.dense3(x)

        return 5 * x  # Scale to -5 to 5 range

    def _init_weights(self):
        # Xavier/Glorot initialization for linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)

In [3]:
class CNN1DModel(nn.Module):
    def __init__(self, num_numerical_features, n_target,
                 symbol_size=39, f09_size=83, f10_size=13,
                 f11_size=540, time_size=967):
        super().__init__()

        # 임베딩 레이어 초기화 (크기 수정)
        self.symbol_embedding = nn.Embedding(symbol_size, 8)
        self.feature_09_embedding = nn.Embedding(f09_size, 4)
        self.feature_10_embedding = nn.Embedding(f10_size, 4)
        self.feature_11_embedding = nn.Embedding(f11_size, 4)
        self.time_embedding = nn.Embedding(time_size, 16)

        # First dense layer to get to 1024 channels
        total_embedding_dim = 8 + 4 + 4 + 4 + 16  # embedding dimensions
        self.input_dim = total_embedding_dim + num_numerical_features
        self.dense1 = nn.Linear(self.input_dim, 258)

        # Convolution blocks
        self.conv1 = nn.Sequential(
            Conv1dBlock(128, 64, kernel_size=5),
        )
        self.avg_pool = nn.AvgPool1d(kernel_size=4, stride=1)

        self.conv2 = nn.Sequential(
            Conv1dBlock(64, 32, kernel_size=3),
        )
        self.max_pool = nn.MaxPool1d(kernel_size=4, stride=1)


        # Final dense layers
        self.dense2 = nn.Linear(6976, 1024)  # //8은 pooling의 총 효과
        self.dense3 = nn.Linear(1024, n_target)
        self.tanh = nn.Tanh()  # tanh 활성화 함수 추가

    def forward(self, numerical_features, symbol, feature_09, feature_10, feature_11, time):
        # Embedding 처리
        symbol_emb = self.symbol_embedding(symbol)           # (batch_size, 8)
        f09_emb = self.feature_09_embedding(feature_09)     # (batch_size, 4)
        f10_emb = self.feature_10_embedding(feature_10)     # (batch_size, 4)
        f11_emb = self.feature_11_embedding(feature_11)     # (batch_size, 4)
        time_emb = self.time_embedding(time)                # (batch_size, 16)

        # 모든 특성을 concatenate
        x = torch.cat([
            numerical_features,
            symbol_emb,
            f09_emb,
            f10_emb,
            f11_emb,
            time_emb
        ], dim=1)  # (batch_size, total_features)

        # 각 feature를 독립적으로 1024 차원으로 확장
        x = x.unsqueeze(-1)  # (batch_size, total_features, 1)
        x = self.dense1(x)   # (batch_size, total_features, 1024)
        x = x.transpose(1, 2)  # (batch_size, 1024, total_features)

        # CNN 처리
        x = self.conv1(x)
        x = self.avg_pool(x)
        x = self.conv2(x)
        x = self.max_pool(x)

        # Dense layers
        x = x.flatten(1)
        x = self.dense2(x)
        x = self.dense3(x)

        # tanh 활성화 함수를 적용하고 5를 곱해 -5~5 범위로 조정
        x = 5 * self.tanh(x)

        return x

In [4]:
class DNNModel(nn.Module):
    def __init__(self, num_numerical_features, n_target,
                 symbol_size=39, f09_size=83, f10_size=13,
                 f11_size=540, time_size=967):
        super().__init__()

        # 임베딩 레이어 초기화
        self.symbol_embedding = nn.Embedding(symbol_size, 8)
        self.feature_09_embedding = nn.Embedding(f09_size, 4)
        self.feature_10_embedding = nn.Embedding(f10_size, 4)
        self.feature_11_embedding = nn.Embedding(f11_size, 4)
        self.time_embedding = nn.Embedding(time_size, 16)

        # Calculate total input size
        total_embedding_dim = 8 + 4 + 4 + 4 + 16  # embedding dimensions
        total_input_size = total_embedding_dim + num_numerical_features

        # Dense layers with batch normalization and dropout
        self.dense_layers = nn.Sequential(
            nn.Linear(total_input_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.01),
        )

        # Final output layer
        self.output_layer = nn.Linear(64, n_target)
        self.tanh = nn.Tanh()

    def forward(self, numerical_features, symbol, feature_09, feature_10, feature_11, time):
        # Embedding 처리
        symbol_emb = self.symbol_embedding(symbol)
        f09_emb = self.feature_09_embedding(feature_09)
        f10_emb = self.feature_10_embedding(feature_10)
        f11_emb = self.feature_11_embedding(feature_11)
        time_emb = self.time_embedding(time)

        # 모든 특성을 concatenate
        x = torch.cat([
            numerical_features,
            symbol_emb,
            f09_emb,
            f10_emb,
            f11_emb,
            time_emb
        ], dim=1)

        # Dense layers
        x = self.dense_layers(x)
        x = self.output_layer(x)

        # Apply tanh and scale to [-5, 5] range
        x = 5 * self.tanh(x)

        return x

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, data, numerical_columns, target_columns, weight_columns=None):
        """
        Args:
            data: pandas DataFrame containing all features
            numerical_columns: list of column names for numerical features
            target_columns: list of target column names
            weight_columns: list of weight column names (optional)
        """
        self.numerical_features = torch.FloatTensor(data[numerical_columns].values)
        self.symbol = torch.LongTensor(data['symbol_id'].values)
        self.feature_09 = torch.LongTensor(data['feature_09'].values)
        self.feature_10 = torch.LongTensor(data['feature_10'].values)
        self.feature_11 = torch.LongTensor(data['feature_11'].values)
        self.time = torch.LongTensor(data['time_id'].values)

        # Multi-target 처리
        self.targets = torch.FloatTensor(data[target_columns].values)

        # 가중치 처리 (옵션)
        if weight_columns:
            self.weights = torch.FloatTensor(data[weight_columns].values)
        else:
            self.weights = torch.ones_like(self.targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'numerical_features': self.numerical_features[idx],
            'symbol_id': self.symbol[idx],
            'feature_09': self.feature_09[idx],
            'feature_10': self.feature_10[idx],
            'feature_11': self.feature_11[idx],
            'time_id': self.time[idx],
            'targets': self.targets[idx],
            'weights': self.weights[idx]
        }

def create_data_loaders(train_data, valid_data, numerical_columns,
                        target_columns, weight_columns=None,
                        batch_size=256, num_workers=4):
    """
    데이터로더를 생성하는 함수

    Args:
        train_data: 학습 데이터가 담긴 DataFrame
        valid_data: 검증 데이터가 담긴 DataFrame
        numerical_columns: 수치형 특성들의 컬럼명 리스트
        target_columns: 타겟 변수들의 컬럼명 리스트
        weight_columns: 가중치 컬럼명 리스트 (옵션)
        batch_size: 배치 크기
        num_workers: 데이터 로딩에 사용할 워커 수
    """

    # Dataset 객체 생성
    train_dataset = CustomDataset(train_data, numerical_columns, target_columns, weight_columns)
    valid_dataset = CustomDataset(valid_data, numerical_columns, target_columns, weight_columns)

    # DataLoader 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, valid_loader


def train_epoch(model, train_loader, valid_loader, optimizer, device):
    model.train()
    total_loss = 0
    total_r2 = 0
    num_batches = len(train_loader)

    for batch_idx, batch in enumerate(train_loader):
        # 데이터를 디바이스로 이동
        numerical_features = batch['numerical_features'].to(device)
        symbol = batch['symbol_id'].to(device)
        feature_09 = batch['feature_09'].to(device)
        feature_10 = batch['feature_10'].to(device)
        feature_11 = batch['feature_11'].to(device)
        time = batch['time_id'].to(device)
        targets = batch['targets'].to(device)
        weights = batch['weights'].to(device)

        # Forward pass
        outputs = model(numerical_features, symbol, feature_09, feature_10, feature_11, time)

        # 손실과 R2 score 계산
        loss = weighted_mse_loss(targets, outputs, weights)
        r2 = weighted_r2_score(targets, outputs, weights)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_r2 += r2

        # 배치별 진행상황 출력 (10배치마다)
        if (batch_idx + 1) % 1000 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            avg_r2 = total_r2 / (batch_idx + 1)
            print(f'Batch [{batch_idx+1}/{num_batches}] Loss: {avg_loss:.4f}, R2: {avg_r2:.4f}')

    return total_loss / num_batches, total_r2 / num_batches

def validate(model, valid_loader, device):
    model.eval()
    total_loss = 0
    total_r2 = 0
    num_batches = len(valid_loader)

    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_loader):
            numerical_features = batch['numerical_features'].to(device)
            symbol = batch['symbol_id'].to(device)
            feature_09 = batch['feature_09'].to(device)
            feature_10 = batch['feature_10'].to(device)
            feature_11 = batch['feature_11'].to(device)
            time = batch['time_id'].to(device)
            targets = batch['targets'].to(device)
            weights = batch['weights'].to(device)

            outputs = model(numerical_features, symbol, feature_09, feature_10, feature_11, time)

            loss = weighted_mse_loss(targets, outputs, weights)
            r2 = weighted_r2_score(targets, outputs, weights)

            total_loss += loss.item()
            total_r2 += r2

        if (batch_idx + 1) % 1000 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            avg_r2 = total_r2 / (batch_idx + 1)
            print(f'Batch [{batch_idx+1}/{num_batches}] Loss: {avg_loss:.4f}, R2: {avg_r2:.4f}')

    return total_loss / num_batches, total_r2 / num_batches

In [6]:
def weighted_mse_loss(y_true, y_pred, weights):
    """
    Multi-target weighted MSE loss

    Args:
        y_true: target values (batch_size, n_targets)
        y_pred: predicted values (batch_size, n_targets)
        weights: weights for each target (batch_size, n_targets)
    """
    return torch.mean(weights * (y_true - y_pred)**2)

def weighted_r2_score(y_true, y_pred, weights):
    """
    Multi-target weighted R2 score

    Args:
        y_true: target values (batch_size, n_targets)
        y_pred: predicted values (batch_size, n_targets)
        weights: weights for each target (batch_size, n_targets)

    Returns:
        weighted R2 score (scalar)
    """
    # Ensure inputs are on CPU and converted to numpy
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    weights = weights.detach().cpu().numpy()

    weights = np.repeat(weights, y_true.shape[1], axis=1)

    # print(y_true.shape, y_pred.shape, weights.shape)
    # Calculate weighted means for each target
    weighted_mean = np.average(y_true, weights=weights, axis=0)

    # Calculate total sum of squares
    total_ss = np.sum(weights * (y_true - weighted_mean) ** 2, axis=0)

    # Calculate residual sum of squares
    residual_ss = np.sum(weights * (y_true - y_pred) ** 2, axis=0)

    # Calculate R2 score for each target
    r2_scores = 1 - (residual_ss / total_ss)

    # Return mean R2 score across all targets
    return np.mean(r2_scores)

In [7]:
def read_parquet_partitions(base_path: str, partitions: int = 10) -> pd.DataFrame:
    dfs = []
    for partition in range(partitions):
        path = f'{base_path}/partition_id={partition}/part-0.parquet'
        df = pd.read_parquet(path)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

# Usage
data = read_parquet_partitions('./data/train.parquet')

In [8]:
data.dropna(inplace=True)

In [9]:
data_length = data.shape[0]

train_ratio = 0.9
split_point = int(data_length * train_ratio)

train_df = data[:split_point]
valid_df = data[split_point:]

print(f"train_df Length: {train_df.shape[0]}")
print(f"valid_df Length: {valid_df.shape[0]}")

train_df Length: 31833739
valid_df Length: 3537083


In [10]:
category_mappings = {'feature_09': {2: 0, 4: 1, 9: 2, 11: 3, 12: 4, 14: 5, 15: 6, 25: 7, 26: 8, 30: 9, 34: 10, 42: 11, 44: 12, 46: 13, 49: 14, 50: 15, 57: 16, 64: 17, 68: 18, 70: 19, 81: 20, 82: 21},
                     'feature_10': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 10: 7, 12: 8},
                     'feature_11': {9: 0, 11: 1, 13: 2, 16: 3, 24: 4, 25: 5, 34: 6, 40: 7, 48: 8, 50: 9, 59: 10, 62: 11, 63: 12, 66: 13,
                                    76: 14, 150: 15, 158: 16, 159: 17, 171: 18, 195: 19, 214: 20, 230: 21, 261: 22, 297: 23, 336: 24, 376: 25, 388: 26, 410: 27, 522: 28, 534: 29, 539: 30},
                     'symbol_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19,
                                   20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38},
                     'time_id' : {i : i for i in range(968)}}

In [11]:
def apply_category_mappings(df, category_mappings):
    """
    주어진 매핑 딕셔너리를 사용하여 범주형 변수들을 변환하는 함수

    Args:
        df: 변환할 데이터프레임
        category_mappings: 각 컬럼별 매핑 딕셔너리
    """
    df = df.copy()  # 원본 데이터 보존

    for column, mapping in category_mappings.items():
        # 매핑되지 않은 값이 있는지 체크
        unmapped_values = set(df[column].unique()) - set(mapping.keys())
        if unmapped_values:
            print(f"Warning: {column}에서 매핑되지 않은 값 발견: {unmapped_values}")

        # 매핑 적용
        df[column] = df[column].map(mapping)

        # NA 값 체크
        na_count = df[column].isna().sum()
        if na_count > 0:
            print(f"Warning: {column}에서 {na_count}개의 NA 값 발견")

    return df

# 데이터로더 생성 전에 매핑 적용
train_df = apply_category_mappings(train_df, category_mappings)
valid_df = apply_category_mappings(valid_df, category_mappings)

# 매핑 후 값 범위 확인
for column in category_mappings.keys():
    print(f"\n{column} range after mapping:")
    print(f"Train: {train_df[column].min()} to {train_df[column].max()}")
    print(f"Valid: {valid_df[column].min()} to {valid_df[column].max()}")


feature_09 range after mapping:
Train: 0 to 21
Valid: 0 to 21

feature_10 range after mapping:
Train: 0 to 8
Valid: 0 to 8

feature_11 range after mapping:
Train: 0 to 30
Valid: 0 to 30

symbol_id range after mapping:
Train: 0 to 38
Valid: 0 to 38

time_id range after mapping:
Train: 68 to 967
Valid: 68 to 967


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
# 컬럼 정의
numerical_columns = data.columns[data.columns.str.contains('feature')]
target_columns = ['responder_0', 'responder_3', 'responder_6']  # 예측할 타겟들
weight_columns = ['weight']  # 각 타겟에 대한 가중치 (옵션)

# 데이터로더 생성
train_loader, valid_loader = create_data_loaders(
    train_data=train_df,
    valid_data=valid_df,
    numerical_columns=numerical_columns,
    target_columns=target_columns,
    weight_columns=weight_columns,
    batch_size=2048
)

In [14]:
# 모델 초기화
model = DNNModel(
    num_numerical_features=len(numerical_columns),
    n_target=len(target_columns),
    symbol_size=39,    # 0-38: 39개
    f09_size=22,      # 0-21: 22개
    f10_size=9,       # 0-8: 9개
    f11_size=31,      # 0-30: 31개
    time_size=968     # 0-967: 968개
).to(device)

In [15]:
# 학습 루프 수정
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    train_loss, train_r2 = train_epoch(model, train_loader, valid_loader, optimizer, device)
    valid_loss, valid_r2 = validate(model, valid_loader, device)

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train R2: {train_r2:.4f}')
    print(f'Valid Loss: {valid_loss:.4f}, Valid R2: {valid_r2:.4f}')
    print('-' * 50)

Batch [1000/15544] Loss: 0.9055, R2: -0.0247
Batch [2000/15544] Loss: 0.8758, R2: 0.0151
Batch [3000/15544] Loss: 0.8617, R2: 0.0331
Batch [4000/15544] Loss: 0.8531, R2: 0.0439
Batch [5000/15544] Loss: 0.8461, R2: 0.0516
Batch [6000/15544] Loss: 0.8411, R2: 0.0577
Batch [7000/15544] Loss: 0.8370, R2: 0.0625
Batch [8000/15544] Loss: 0.8338, R2: 0.0663
Batch [9000/15544] Loss: 0.8308, R2: 0.0697
Batch [10000/15544] Loss: 0.8286, R2: 0.0723
Batch [11000/15544] Loss: 0.8267, R2: 0.0747
Batch [12000/15544] Loss: 0.8251, R2: 0.0769
Batch [13000/15544] Loss: 0.8236, R2: 0.0787
Batch [14000/15544] Loss: 0.8221, R2: 0.0804
Batch [15000/15544] Loss: 0.8209, R2: 0.0819
Epoch 1/5:
Train Loss: 0.8201, Train R2: 0.0826
Valid Loss: 1.1504, Valid R2: -1.3336
--------------------------------------------------
Batch [1000/15544] Loss: 0.8012, R2: 0.1028
Batch [2000/15544] Loss: 0.8009, R2: 0.1040
Batch [3000/15544] Loss: 0.8009, R2: 0.1050
Batch [4000/15544] Loss: 0.8005, R2: 0.1052
Batch [5000/15544] L

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x704a96688a70>>
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 